In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim)
        self.hidden2out = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)


    def forward(self, x):
        _, lstm_out = self.lstm(x.view(len(x), 1, -1))
        class_space = self.hidden2out(lstm_out[0].view(-1, self.hidden_dim))
        class_scores = self.softmax(class_space)
        return class_scores

In [19]:
def sample_series(target: int):
    mean = target * 5 + 2
    var = target / 0.5
    x = np.random.normal(mean + 0.5, var + 0.1, size=100)
    y = np.random.normal(mean - 0.5, var + 0.1, size=100)
    return torch.tensor([x, y]).float().T

In [22]:
# 時系列の2値分類
lstm_classifier = LSTMClassifier(
    input_dim=2,
    hidden_dim=16,
    output_dim=2
)

In [23]:
# 学習前なのでランダムな確率値
lstm_classifier.forward(sample_series(0))

tensor([[0.5950, 0.4050]], grad_fn=<SoftmaxBackward0>)

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_classifier.parameters(), lr=0.001)

In [27]:
datasets = []
for i in range(100):
    target = torch.tensor([i%2])
    inputs = sample_series(i % 2)
    datasets.append((inputs, target))

In [28]:
verbose = 100

for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, target) in enumerate(datasets):
        optimizer.zero_grad()
        outputs = lstm_classifier(inputs)
        loss = criterion(outputs, target)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % verbose == verbose-1:
            print(f"epoch:{epoch + 1}, batch:{i + 1}, loss: {running_loss / verbose}")
            running_loss = 0.0

epoch:1, batch:100, loss: 0.5975463700294494
epoch:2, batch:100, loss: 0.3630118614435196
epoch:3, batch:100, loss: 0.3274684479832649
epoch:4, batch:100, loss: 0.3211913686990738
epoch:5, batch:100, loss: 0.31845935970544814
epoch:6, batch:100, loss: 0.3168721160292625
epoch:7, batch:100, loss: 0.3158703327178955
epoch:8, batch:100, loss: 0.3152585390210152
epoch:9, batch:100, loss: 0.31486267685890196
epoch:10, batch:100, loss: 0.31458419501781465


In [29]:
lstm_classifier(sample_series(0))

tensor([[9.9908e-01, 9.1740e-04]], grad_fn=<SoftmaxBackward0>)

In [30]:
lstm_classifier(sample_series(1))

tensor([[0.0033, 0.9967]], grad_fn=<SoftmaxBackward0>)